## **IMPORING THE REQUIRED LIBARIES**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

## **READING DATASET AND PRE-PROCESSING OF DATA**

# Reading Dataset

In [ ]:
cd /content/drive/MyDrive/IBM

/content/drive/MyDrive/IBM


In [ ]:
df = pd.read_csv("spam.csv",encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Pre - Processing Data

In [ ]:
#Removing null values
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1, inplace = True)

In [ ]:
le = LabelEncoder()
df["v1"] = le.fit_transform(df["v1"])

y = df['v1'].values
y = y.reshape(-1,1)

df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
df1 = []

In [ ]:
#pre processing sentences and splitting them. Followed by stemming.
for i in range(0,5572):
  v2 = df['v2'][i]
  v2 = re.sub('[^a-zA-Z]', ' ', v2) 
  v2 = v2.lower()
  v2 = v2.split()
  v2 = [ ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]
  v2 = ' '.join(v2)
  df1.append(v2)

df1 = np.array(df1)
df1.shape

(5572,)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000)
x = cv.fit_transform(df1).toarray()

In [ ]:
#splitting train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

## **CREATE MODEL**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

## **ADDING LAYERS**

In [ ]:
model.add(LSTM(units = 50, input_shape = (x_train.shape[1],1), return_sequences = True))
model.add(LSTM(units = 50, return_sequences = True))
model.add(LSTM(units = 50, return_sequences = True))
model.add(LSTM(units = 50))
model.add(Dense(units = 1))

## **COMPILE THE MODEL**

In [ ]:
model.compile(optimizer = 'adam', loss = 'mse')

## **FIT THE MODEL**

In [16]:
model.fit(x_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
122/122 [==============================] - 513s 4s/step - loss: 0.1223
Epoch 2/10
122/122 [==============================] - 501s 4s/step - loss: 0.1145
Epoch 3/10
122/122 [==============================] - 497s 4s/step - loss: 0.1143
Epoch 4/10
122/122 [==============================] - 500s 4s/step - loss: 0.1144
Epoch 5/10
122/122 [==============================] - 498s 4s/step - loss: 0.1142
Epoch 6/10
122/122 [==============================] - 498s 4s/step - loss: 0.1144
Epoch 7/10
122/122 [==============================] - 499s 4s/step - loss: 0.1142
Epoch 8/10
122/122 [==============================] - 493s 4s/step - loss: 0.1139
Epoch 9/10
122/122 [==============================] - 499s 4s/step - loss: 0.1140
Epoch 10/10
122/122 [==============================] - 500s 4s/step - loss: 0.1138


## **SAVE THE MODEL**

In [18]:
model.save("spam-message-classifier.h3")

## **TEST THE MODEL**

In [19]:
#SPAM MESSAGE

text = "WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."
text = re.sub('[^a-zA-Z]',' ',text)
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english')) ]
text = ' '.join(text)

text

'winner as valu network custom select receivea prize reward to claim call claim code kl valid hour'

In [20]:
text1 = cv.transform([text])
text1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [21]:
from tensorflow.keras.models import load_model
model1 = load_model('spam-message-classifier.h1')

In [ ]:
pred = model1.predict(text1.astype(float))